# EDA on given Formula 1 dataset using SQL

Queries covered in this notebook:

1. Finding locations that hosted maximum number of race
2. Finding circuits that hosted opening races
3. Number of races hosted by every country
4. All time constructor points and leaders
5. Number of races organized per season
6. Comparison of all time top 10 teams (based on points and races won)
7. Number of races won by top 10 drivers
8. Finding the fastest laps on each circuit
9. Number of constructors from each country
10. Finding the fastest quali times on every circuit for pole position
11. Drivers who have won atleast one race

In [1]:
import pandas as pd
import sqlite3 
import plotly.express as px
import plotly.graph_objects as go
import os

In [2]:
# Establishing a connection between the SQLITE3 database
conn = sqlite3.connect("f1.db")

In [ ]:
# Reading CSV files

data_1 = pd.read_csv("data/circuits.csv")
data_2 = pd.read_csv("data/constructor_results.csv")
data_3 = pd.read_csv("data/constructor_standings.csv")
data_4 = pd.read_csv("data/constructors.csv")
data_5 = pd.read_csv("data/driver_standings.csv")
data_6 = pd.read_csv("data/drivers.csv")
data_7 = pd.read_csv("data/lap_times.csv")
data_8 = pd.read_csv("data/pit_stops.csv")
data_9 = pd.read_csv("data/qualifying.csv")
data_10 = pd.read_csv("data/races.csv")
data_11 = pd.read_csv("data/seasons.csv")
data_12 = pd.read_csv("data/status.csv")

# Inserting data into SQL Tables

data_1.to_sql("circuits", conn)
data_2.to_sql("constructor_results", conn)
data_3.to_sql("constructor_standings", conn)
data_4.to_sql("constructors", conn)
data_5.to_sql("driver_standings", conn)
data_6.to_sql("drivers", conn)
data_8.to_sql("pit_stops", conn)
data_9.to_sql("qualifying", conn)
data_10.to_sql("races", conn)
data_11.to_sql("seasons", conn)
data_12.to_sql("status", conn)

In [13]:
data_13 = pd.read_csv("data/results.csv")
data_13.to_sql("results", conn)

In [38]:
data_7.to_sql("lap_times", conn)

In [22]:
# Finding the locations that hosted the maximum races

cir_max = pd.read_sql('select c.location as "City", count(r.circuitid) as "Races Hosted" from circuits c, races r where r.circuitid = c.circuitid group by "City" order by "Races Hosted" desc;', conn)
cir_max

px.bar(cir_max, x="City", y="Races Hosted", color="Races Hosted", title="Races hosted by cities")

In [3]:
# Finding the circuits that have hosted the opening races

op_ci = pd.read_sql('select r.name as "Grand Prix Name", c.name as "Circuit Name", c.location as "City", c.country as "Country", count(*) as "Opening Races Hosted" from circuits c, races r where r.circuitid = c.circuitid and r.round = 1 group by "City" order by "Opening Races Hosted" desc;', conn)
op_ci

,Grand Prix Name,Circuit Name,City,Country,Opening Races Hosted
0,Australian Grand Prix,Albert Park Grand Prix Circuit,Melbourne,Australia,22
1,Argentine Grand Prix,Autódromo Juan y Oscar Gálvez,Buenos Aires,Argentina,15
2,South African Grand Prix,Kyalami,Midrand,South Africa,8
3,Brazilian Grand Prix,Autódromo Internacional Nelson Piquet,Rio de Janeiro,Brazil,7
4,Monaco Grand Prix,Circuit de Monaco,Monte-Carlo,Monaco,5
5,Brazilian Grand Prix,Autódromo José Carlos Pace,São Paulo,Brazil,3
6,Bahrain Grand Prix,Bahrain International Circuit,Sakhir,Bahrain,3
7,United States Grand Prix,Phoenix street circuit,Phoenix,USA,2
8,Swiss Grand Prix,Circuit Bremgarten,Bern,Switzerland,2
9,Dutch Grand Prix,Circuit Park Zandvoort,Zandvoort,Netherlands,1


In [27]:
px.bar(op_ci, x="City", y="Opening Races Hosted", color="Opening Races Hosted", title="Cities that hosted opening races in decade 2010-2021")

In [31]:
# Number of races hosted by every country

cont_ra = pd.read_sql('select c.country as "Country", count(*) as "Races Hosted" from circuits c, races r where r.circuitid = c.circuitid group by "Country" order by "Races Hosted" desc;', conn)
cont_ra

,Country,Races Hosted
0,Italy,102
1,Germany,79
2,UK,76
3,USA,71
4,Monaco,67
5,Belgium,66
6,France,62
7,Spain,58
8,Canada,50
9,Brazil,48


In [32]:
px.bar(cont_ra, x="Country", y="Races Hosted", color="Races Hosted", title="Countries that have hosted race")

In [17]:
# All time constructor points and leaders

con_at = pd.read_sql('select c.name as "Constructor", c.nationality as "Nationality", sum(points) as "Total Points" from constructor_results r, constructors c where c.constructorId = r.constructorId group by "Constructor" order by "Total Points" desc;', conn)
con_at

,Constructor,Nationality,Total Points
0,Ferrari,Italian,8558.5
1,Mercedes,German,5927.0
2,McLaren,British,5795.5
3,Red Bull,Austrian,5329.5
4,Williams,British,3567.0
...,...,...,...
170,Behra-Porsche,Italian,0.0
171,BRM-Ford,British,0.0
172,Apollon,Swiss,0.0
173,Andrea Moda,Italian,0.0


In [26]:
px.bar(con_at, x="Constructor", y="Total Points", color="Total Points", title="All time points scored by constructors")

In [50]:
# Finding number of races organized per season

ra_season = pd.read_sql('select strftime("%Y", "date") as "Year", count(*) as "Races held" from races group by "Year" order by "Year";', conn)
ra_season

,Year,Races held
0,1950,7
1,1951,8
2,1952,8
3,1953,9
4,1954,9
...,...,...
67,2017,20
68,2018,21
69,2019,21
70,2020,17


In [57]:
px.line(ra_season, x="Year", y="Races held", title="Races held every season")

In [9]:
# Comparing the all-time top 5 teams

con_top = pd.read_sql('select c.name as "Constructor", c.nationality as "Nationality", sum(points) as "Total Points" from constructor_results r, constructors c where c.constructorId = r.constructorId and points > 0 group by "Constructor" order by "Total Points" desc limit 5;', conn)
con_top

,Constructor,Nationality,Total Points
0,Ferrari,Italian,8558.5
1,Mercedes,German,5927.0
2,McLaren,British,5795.5
3,Red Bull,Austrian,5329.5
4,Williams,British,3567.0


In [23]:
# Finding the number of races won by the top 5 teams

raw_top = pd.read_sql('select c.name as "Constructor", count(r.constructorId) as "Races won" from results r, constructors c where c.constructorId = r.constructorId and "Constructor" in ("Ferrari", "Mercedes", "McLaren", "Red Bull", "Williams") and r.position = 1 group by "Constructor" order by "Races won" desc;', conn)
raw_top

,Constructor,Races won
0,Ferrari,239
1,McLaren,178
2,Mercedes,118
3,Williams,114
4,Red Bull,70


In [4]:
# Finding the number of races won by top 10 drivers

rawd_top = pd.read_sql('select ("forename" || " " || "surname") as "Name", count(r.driverId) as "Races Won" from drivers d, results r where r.driverId = d.driverId and r.position = 1 group by "Name" order by "Races Won" desc limit 10;', conn)
rawd_top

,Name,Races Won
0,Lewis Hamilton,98
1,Michael Schumacher,91
2,Sebastian Vettel,53
3,Alain Prost,51
4,Ayrton Senna,41
5,Fernando Alonso,32
6,Nigel Mansell,31
7,Jackie Stewart,27
8,Niki Lauda,25
9,Jim Clark,25


In [33]:
# Finding the fastest laps on each circuit

fas_lap = pd.read_sql('select c.location as "City", r.year as "Year of race", min(l.time) as "Lap Time", ("forename" || " " || "surname") as "Driver" from circuits c, lap_times l, races r, drivers d where l.raceId = r.raceId and r.circuitId = c.circuitId and l.driverId = d.driverId group by "City" order by "City";', conn)
fas_lap

,City,Year of race,Lap Time,Driver
0,Abu Dhabi,2019,1:39.283,Lewis Hamilton
1,Austin,2019,1:36.169,Charles Leclerc
2,Baku,2019,1:43.009,Charles Leclerc
3,Budapest,2001,19:29.677,Olivier Panis
4,Buenos Aires,1996,15:41.618,Mika Salo
5,Estoril,1996,1:22.873,Jacques Villeneuve
6,Hockenheim,2005,14:06.799,Mark Webber
7,Imola,2006,13:29.130,Yuji Ide
8,Indianapolis,2004,1:10.399,Rubens Barrichello
9,Istanbul,2005,1:24.770,Juan Pablo Montoya


In [10]:
# Constructors from each country

con_na = pd.read_sql('select nationality as "Nationality", count(*) as "Number of constructors" from constructors group by "Nationality" order by "Number of constructors" desc;', conn)
con_na

,Nationality,Number of constructors
0,British,86
1,American,39
2,Italian,29
3,French,13
4,German,10
5,Swiss,5
6,Japanese,5
7,South African,3
8,Dutch,3
9,Russian,2


In [11]:
px.bar(con_na, x="Nationality", y="Number of constructors", color="Number of constructors", title="Number of constructors from each country")

In [13]:
# Fastest quali times for pole position for every race

fa_qpo = pd.read_sql('select r.name as "Grand Prix Name", ("forename" || " " || "surname") as "Driver", min(q1) as "Fastest Q1 time", min(q2) as "Fasstes Q2 time", min(Q3) as "Fastest Q3 time" from qualifying q, races r, drivers d where q.raceId = r.raceId and q.position = 1 and q.driverId = d.driverId group by "Grand Prix Name" order by "Grand Prix Name";', conn)
fa_qpo

,Grand Prix Name,Driver,Fastest Q1 time,Fasstes Q2 time,Fastest Q3 time
0,70th Anniversary Grand Prix,Valtteri Bottas,1:26.738,1:25.785,1:25.154
1,Abu Dhabi Grand Prix,Lewis Hamilton,1:35.851,1:35.634,1:34.779
2,Argentine Grand Prix,David Coulthard,1:24.473,\N,\N
3,Australian Grand Prix,Lewis Hamilton,1:16.179,1:21.014,1:20.486
4,Austrian Grand Prix,Valtteri Bottas,1:04.111,1:03.015,1:02.939
5,Azerbaijan Grand Prix,Valtteri Bottas,1:41.983,1:41.275,1:40.495
6,Bahrain Grand Prix,Lewis Hamilton,1:28.343,1:27.586,1:27.264
7,Belgian Grand Prix,Lewis Hamilton,1:42.323,1:41.553,1:41.252
8,Brazilian Grand Prix,Lewis Hamilton,1:08.242,1:07.503,1:07.281
9,British Grand Prix,Sebastian Vettel,1:18.233,1:18.119,1:19.509


In [50]:
# Finding driver who have won at least one race

ra_won = pd.read_sql('select ("forename" || " " || "surname") as "Driver", sum(case when position = 1 then 1 else 0 end) as "Races Won" from drivers d, results r where r.driverId = d.driverId and position = 1 group by "Driver" order by "Races Won" desc;', conn)
ra_won

,Driver,Races Won
0,Lewis Hamilton,98
1,Michael Schumacher,91
2,Sebastian Vettel,53
3,Alain Prost,51
4,Ayrton Senna,41
...,...,...
105,Giancarlo Baghetti,1
106,François Cevert,1
107,Carlos Pace,1
108,Bob Sweikert,1


In [51]:
px.bar(ra_won, x="Driver", y="Races Won", color="Races Won", title="Drivers who have won the race")